## Test with straight beam

The aim of this test is to compare the inertial loads of an analytical case with the ones given by the code.

The beam used is straight, measures 1 meter, weighs 100 kg and it only has two nodes. It is defined with the following half chord position and property files.


The stiffness matrix is not needed in this test, but, to make the structural code work, a valid matrix was used from the first node of the original IEA 15 MW reference WT file.

In [362]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from beam_corot.ComplBeam import ComplBeam
from beam_corot.CoRot import CoRot
from utils import save_load
from inertial_forces_v2 import inertial_loads_fun, inertial_loads_fun_v02, inertial_loads_fun_v03


In [363]:
# Model input json file name
f_model_json = "straight_beam_no_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

pos = np.concatenate((beam_no_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [364]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_no_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.000000,0.042634,4.209301,-0.093673,16.499820,0.207198,0.000000,0.042634,-4.124033,0.093673
1,-0.207198,33.580883,0.000000,-4.229476,-0.062810,-0.137028,0.207198,16.419117,0.000000,4.103857,-0.062810,0.137028
2,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000
3,0.042634,-4.229476,0.000000,0.834256,0.000922,0.000751,-0.042634,-4.103857,0.000000,-0.832411,0.000922,-0.000751
4,4.209301,-0.062810,0.000000,0.000922,0.834256,0.000751,4.124033,0.062810,0.000000,0.000922,-0.832411,-0.000751
5,-0.093673,-0.137028,0.000000,0.000751,0.000751,0.004702,0.093673,0.137028,0.000000,0.000751,0.000751,-0.004702
6,16.499820,0.207198,0.000000,-0.042634,4.124033,0.093673,33.500180,-0.207198,0.000000,-0.042634,-4.209301,-0.093673
7,0.207198,16.419117,0.000000,-4.103857,0.062810,0.137028,-0.207198,33.580883,0.000000,4.229476,0.062810,-0.137028
8,0.000000,0.000000,16.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
9,0.042634,4.103857,0.000000,-0.832411,0.000922,0.000751,-0.042634,4.229476,0.000000,0.834256,0.000922,-0.000751


## Test 1
No CoG offset, no pitch angle, no hub radius.

The operational condicions are:

    RPM = 1 rad/s
    pitch = 0

The centrifugar force in its center of mass is $F=m\cdot \omega^{2}\cdot r= 100 \cdot 1 \cdot 0.5 = 50 \text{ N}$.

In [365]:
# Operations info
omega = 1 # rad/s

In [366]:
# Calculate inertial loads using the CoG accelerations (positions in our input)
offset = np.zeros((2,3))

load_t1 = inertial_loads_fun_v03(pos,offset,M_mat_no_offset,0,omega,0)
load_t1 = np.reshape(load_t1, (-1,6))

force, moment = np.split(load_t1,[3],axis=1)
force_df = pd.DataFrame(force, columns=['X', 'Y', 'Z'])
force_df


,X,Y,Z
0,0.0,0.0,-16.666667
1,0.0,0.0,-33.333333


In [367]:
moment_df = pd.DataFrame(moment, columns=['X', 'Y', 'Z'])
moment_df

,X,Y,Z
0,0.0,0.0,0.0
1,0.0,0.0,0.0


In [368]:
force, moment = np.split(load_t1,[3],axis=1)
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')

r_globcg = np.array([0,0,0.5]) # The global CoG
r_globcg2node = pos[:,:3] + offset - r_globcg
print(f'{r_globcg2node = }')

moment_cg = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([  0.,   0., -50.])
r_globcg2node = array([[ 0. ,  0. , -0.5],
       [ 0. ,  0. ,  0.5]])
moment_cg = array([0., 0., 0.])


In [369]:
f_analytical = np.array([0,0,-50])
m_analytical = np.zeros(3)

if np.allclose(force_cg, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_cg, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_cg, f_analytical) and np.allclose(moment_cg, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 1.5

This time the beam is 2 meters long.

In [370]:
# Model input json file name
f_model_json = "straight_beam_2m_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_no_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_no_offset.nodeLocations[:,2] # z-axis position
beam_no_offset.nodeLocations

pos = np.concatenate((beam_no_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 0., 0., 0.]])


In [371]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_2m_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_no_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_no_offset = beam_no_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_no_offset_df = pd.DataFrame(M_mat_no_offset)
M_mat_no_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,67.685220,-1.254818,0.000000,0.540898,17.207565,-0.765513,32.314780,1.254818,0.000000,0.540898,-16.125768,0.765513
1,-1.254818,68.157749,0.000000,-17.443829,-0.777163,-1.062936,1.254818,31.842251,0.000000,15.889504,-0.777163,1.062936
2,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000
3,0.540898,-17.443829,0.000000,6.729910,0.063243,0.030580,-0.540898,-15.889504,0.000000,-6.603424,0.063243,-0.030580
4,17.207565,-0.777163,0.000000,0.063243,6.729910,0.030580,16.125768,0.777163,0.000000,0.063243,-6.603424,-0.030580
5,-0.765513,-1.062936,0.000000,0.030580,0.030580,0.145673,0.765513,1.062936,0.000000,0.030580,0.030580,-0.145673
6,32.314780,1.254818,0.000000,-0.540898,16.125768,0.765513,67.685220,-1.254818,0.000000,-0.540898,-17.207565,-0.765513
7,1.254818,31.842251,0.000000,-15.889504,0.777163,1.062936,-1.254818,68.157749,0.000000,17.443829,0.777163,-1.062936
8,0.000000,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,0.000000,66.666667,0.000000,0.000000,0.000000
9,0.540898,15.889504,0.000000,-6.603424,0.063243,0.030580,-0.540898,17.443829,0.000000,6.729910,0.063243,-0.030580


In [372]:
# Calculate inertial loads using the CoG accelerations (positions in our input)
offset = np.zeros((2,3))

load_t1 = inertial_loads_fun_v03(pos,offset,M_mat_no_offset,0,omega,0)
load_t1 = np.reshape(load_t1, (-1,6))

force, moment = np.split(load_t1,[3],axis=1)
print(f'{force = }')
print(f'{moment = }')

force = array([[   0.        ,    0.        ,  -66.66666667],
       [   0.        ,    0.        , -133.33333333]])
moment = array([[0., 0., 0.],
       [0., 0., 0.]])


In [373]:
force, moment = np.split(load_t1,[3],axis=1)
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')

r_globcg = np.array([0,0,1]) # The global CoG
r_globcg2node = pos[:,:3] + offset - r_globcg
print(f'{r_globcg2node = }')

moment_cg = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([   0.,    0., -200.])
r_globcg2node = array([[ 0.,  0., -1.],
       [ 0.,  0.,  1.]])
moment_cg = array([0., 0., 0.])


In [374]:
f_analytical = np.array([0,0,-200])
m_analytical = np.zeros(3)

if np.allclose(force_cg, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_cg, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_cg, f_analytical) and np.allclose(moment_cg, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
The two results are the same.
Test passed.


## Test 2

There is a 10 cm offset of the CoG in the x-direction.

No hub, no pitch

In [375]:
# Model input json file name
f_model_json = "straight_beam_offset_static.json"

# Input files folder
inputfolder = os.path.join(os.getcwd(),'straight_beam')
mainfile = os.path.join(inputfolder,f_model_json)

# - Instantiate beam
save_load([0], inputfolder, onlyy=True) # Creates a force file
beam_offset = ComplBeam(mainfile)

# - Get the radius location of nodes
r = beam_offset.nodeLocations[:,2] # z-axis position
beam_offset.nodeLocations

pos = np.concatenate((beam_offset.nodeLocations,np.zeros((2,3))), axis=1)
print(f'{pos = }')

-----------
ComplBeam Model Created
Static analysis done
pos = array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])


In [376]:
# - Get the mass matrix
# For non dynamic calculations the mass matrix is not calculated
f_model_json = "straight_beam_offset_dynamic.json"
mainfile_dyn = os.path.join(inputfolder,f_model_json)

# Calculate mass matrix
beam_offset_dyn = ComplBeam(mainfile_dyn)
M_mat_offset = beam_offset_dyn.M_mat_full

# Display DataFrame in Jupyter notebook
M_mat_offset_df = pd.DataFrame(M_mat_offset)
M_mat_offset_df

-----------
**********                  No "mass_matrix" type defined                    **********
**********          "mass_matrix" = "Timo" / "Compl" (default = "Timo")        **********
Timo Mass Matrix
ComplBeam Model Created
Dynamic analysis done


,0,1,2,3,4,5,6,7,8,9,10,11
0,33.500180,-0.207198,0.244735,0.042634,4.209301,-0.105866,16.499820,0.207198,0.244735,0.042634,-4.124033,0.105866
1,-0.207198,33.580883,-0.244735,-4.229476,-0.062810,3.208498,0.207198,16.419117,-0.244735,4.103857,-0.062810,1.791502
2,0.244735,-0.244735,33.333333,0.122368,-3.210966,-0.285525,-0.244735,0.244735,16.666667,0.122368,-1.544299,0.285525
3,0.042634,-4.229476,0.122368,0.834256,0.000922,-0.422012,-0.042634,-4.103857,0.122368,-0.832411,0.000922,-0.411321
4,4.209301,-0.062810,-3.210966,0.000922,0.834256,-0.005346,4.124033,0.062810,-1.544299,0.000922,-0.832411,0.005346
5,-0.105866,3.208498,-0.285525,-0.422012,-0.005346,-0.022404,0.105866,1.791502,-0.285525,0.411321,-0.005346,0.022404
6,16.499820,0.207198,-0.244735,-0.042634,4.124033,0.105866,33.500180,-0.207198,-0.244735,-0.042634,-4.209301,-0.105866
7,0.207198,16.419117,0.244735,-4.103857,0.062810,1.791502,-0.207198,33.580883,0.244735,4.229476,0.062810,3.208498
8,0.244735,-0.244735,16.666667,0.122368,-1.544299,-0.285525,-0.244735,0.244735,33.333333,0.122368,-3.210966,0.285525
9,0.042634,4.103857,0.122368,-0.832411,0.000922,0.411321,-0.042634,4.229476,0.122368,0.834256,0.000922,0.422012


In [377]:
# Calculate inertial loads using the CoG accelerations
offset = np.array([[0.1,0,0],[0.1,0,0]])

load_t2 = inertial_loads_fun_v03(pos,offset,M_mat_offset,0,omega,0)
load_t2 = np.reshape(load_t2, (-1,6))

force, moment = np.split(load_t2,[3],axis=1)
# print(f'{force = }')
# print(f'{moment = }')

In [378]:
force_df = pd.DataFrame(force, columns=['X', 'Y', 'Z'])
print('force')
force_df

force


,X,Y,Z
0,-5.244735,0.244735,-16.666667
1,-4.755265,-0.244735,-33.333333


In [379]:
moment_df = pd.DataFrame(moment, columns=['X', 'Y', 'Z'])
print('moment')
moment_df

moment


,X,Y,Z
0,-0.122368,2.377632,0.309998
1,-0.122368,7.377632,-0.309998


In [380]:
force, moment = np.split(load_t2,[3],axis=1)
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')

r_globcg = np.array([0.1,0,0.5]) # The global CoG
r_globcg2node = pos[:,:3] - r_globcg
print(f'{r_globcg2node = }')

moment_cg = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_cg = }')
print(np.sum(moment,axis=0))
print(np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1]))

force_cg = array([-1.00000000e+01, -2.77555756e-17, -5.00000000e+01])
r_globcg2node = array([[-0.1,  0. , -0.5],
       [-0.1,  0. ,  0.5]])
moment_cg = array([5.20278265e-14, 5.00000000e+00, 3.46944695e-18])
[-0.24473539  9.75526461  0.        ]
[ 2.44735387e-01 -4.75526461e+00  3.46944695e-18]


In [381]:
f_analytical = np.array([-10,0.,-50])
m_analytical = np.zeros(3)

if np.allclose(force_cg, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_cg, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_cg, f_analytical) and np.allclose(moment_cg, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
ERROR: The two results are different.
Test failed.


Since it does not work, let's use the previous function, `inertial_loads_fun_v02`.

In [382]:
# Calculate inertial loads using the CoG accelerations
offset = np.array([[0.1,0,0],[0.1,0,0]])

load_t2_2 = inertial_loads_fun_v02(pos,offset,M_mat_offset,omega)
load_t2_2 = np.reshape(load_t2_2, (-1,6))

force, moment = np.split(load_t2_2,[3],axis=1)
# print(f'{force = }')
# print(f'{moment = }')

In [383]:
force_df = pd.DataFrame(force, columns=['X', 'Y', 'Z'])
print('force')
force_df

force


,X,Y,Z
0,-5.244735,0.244735,-16.666667
1,-4.755265,-0.244735,-33.333333


In [384]:
moment_df = pd.DataFrame(moment, columns=['X', 'Y', 'Z'])
print('moment')
moment_df

moment


,X,Y,Z
0,0.0,0.028552,-0.071097
1,0.0,-0.028552,-0.404430


In [385]:
force, moment = np.split(load_t2_2,[3],axis=1)
force_cg = np.sum(force, axis=0)
print(f'{force_cg = }')

r_globcg = np.array([0.1,0,0.5]) # The global CoG
r_globcg2node = pos[:,:3] - r_globcg
print(f'{r_globcg2node = }')

moment_cg = np.sum(moment,axis=0) + np.cross(r_globcg2node[0],force[0]) + np.cross(r_globcg2node[1],force[1])
print(f'{moment_cg = }')

force_cg = array([-1.00000000e+01, -2.77555756e-17, -5.00000000e+01])
r_globcg2node = array([[-0.1,  0. , -0.5],
       [-0.1,  0. ,  0.5]])
moment_cg = array([ 0.24473539, -4.75526461, -0.47552646])


In [386]:
f_analytical = np.array([-10,0.,-50])
m_analytical = np.zeros(3)

if np.allclose(force_cg, f_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(moment_cg, m_analytical): print('The two results are the same.')
else: print('ERROR: The two results are different.')
if np.allclose(force_cg, f_analytical) and np.allclose(moment_cg, m_analytical):
    print('Test passed.')
else:
    print('Test failed.')

The two results are the same.
ERROR: The two results are different.
Test failed.
